In [1]:
import os
import sys
import json
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())


In [3]:
spark

In [4]:
sc = spark.sparkContext

In [5]:
films = sc.textFile("/labs/laba01/ml-100k/u.item")

In [6]:
films.getNumPartitions()

2

In [7]:
films.count()

1682

In [8]:
rayting = sc.textFile("/labs/laba01/ml-100k/u.data")

In [9]:
rayting.count()

100000

In [10]:
rayting.getNumPartitions()

2

In [11]:
rayting.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [12]:
result = {
   "hist_film": None,
   "hist_all": None
}


In [13]:
ID = '302'
result["hist_film"] = sorted(rayting.map(lambda x: x.split("\t"))
         .filter(lambda x: x[1] == ID)
         .map(lambda x: (x[2], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())
result["hist_film"] = [i[1] for i in result["hist_film"]]
result["hist_film"]


[2, 10, 46, 119, 120]

In [14]:
result["hist_all"] = sorted(rayting.map(lambda x: x.split("\t"))
         .map(lambda x: (x[2], 1))
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())
result["hist_all"] = [i[1] for i in result["hist_all"]]
result["hist_all"]


[6110, 11370, 27145, 34174, 21201]

In [15]:
result

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [16]:
with open('lab01.json', 'w') as f:
    json.dump(result, f)


In [17]:
sc.stop()